# **1. 라이브러리 및 데이터 적재**

In [1]:
import os
import datetime as dt
import math
import copy
import re
from unicodedata import normalize

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
#데이터 적재
sales_2019 = pd.read_excel('../data/internal/raw_data/실적데이터_수정.xlsx', sheet_name = 'rawdata_2019(완)', skiprows = 1) # 2019년 실적데이터, 첫째 행 제외
sales_2020 = pd.read_excel('../data/internal/raw_data/평가데이터.xlsx', sheet_name = '6월편성', skiprows = 1) # 2020년 6월 평가 데이터, 첫째 행 제외

실적 = sales_2019.copy() # 원본 데이터 보존을 위해 각 데이터의 사본 사용
평가 = sales_2020.copy()

In [4]:
print("2019 판매실적 데이터 레코드 수:", len(실적), "행")
print("2020 평가 데이터 레코드 수:", len(평가), "행")

2019 판매실적 데이터 레코드 수: 38309 행
2020 평가 데이터 레코드 수: 2891 행


# **2. 데이터 전처리**

## 1) 내부 데이터

### 1) - 1. 2019년도 실적 데이터 전처리 

In [5]:
def preprocessing_table_2019(실적):
    '''
    판매실적 데이터 전처리 및 파생변수 생성하는 작업 수행 
    '''
  
    #방송일시 변수에서 파생 변수 생성
    
    실적['년'] = 실적.방송일시.apply(lambda X: X.year)
    실적['주차'] = 실적.방송일시.apply(lambda X: X.week)
    실적['월'] = 실적.방송일시.apply(lambda X: X.month)
    실적['일'] = 실적.방송일시.apply(lambda X: X.day)
    실적['요일'] = 실적.방송일시.apply(lambda X: X.weekday()) # 0 - 월요일, 1 - 화요일, ..., 6 - 일요일
    실적['시'] = 실적.방송일시.apply(lambda X: X.hour)
    실적['분'] = 실적.방송일시.apply(lambda X: X.minute)
    실적['주말'] = [1 if i==5 or i==6 else 0 for i in 실적['요일']]
    실적['방송날짜'] = 실적.방송일시.dt.date # '방송일시' 컬럼에서 시/분 제외한 날짜 정보 추출하여 별도의 컬럼 생성
  
    # 2019년도 기준 공휴일 정보를 바탕으로 별도의 컬럼 생성
    # 주말과 겹치는 공휴일도 상관없이 표시
    # 공휴일인 날은 1, 공휴일 아닌 날은 0
    
    실적['공휴일'] = [i.strftime('%Y-%m-%d') for i in 실적.방송날짜]
    
    공휴일 = [
                '2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01',
                '2019-05-05', '2019-05-06', '2019-05-12', '2019-06-06', '2019-08-15',
                '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09',
                '2019-12-25', '2020-01-01'
            ]
    idx = 실적.공휴일.isin(공휴일)
    nidx = ~ 실적.공휴일.isin(공휴일)
    실적.loc[idx,'공휴일'] = 1
    실적.loc[nidx,'공휴일'] = 0

    # '상품명' 기준으로 '지불방식' 에 따른 파생 변수 생성
    
    지불방식 = []
    for i in 실적.상품명:
        if '무이자' in i or '(무)' in i: # 상품명에 '무이자' 라는 표기 있을 시 1 대입
            지불방식.append(1)
        elif '일시불' in i or '(일)' in i: # 상품명에 '일시불' 이라는 표기 있을 시 2 대입
            지불방식.append(2)
        else:
            지불방식.append(0) # 상품명에 지불 방식 관련 표기 없을 시 0 대입
 
    실적['지불방식'] = 지불방식

    #노출(분) 변수로 파생 변수 생성 
    #(1) 노출분 : 노출(분)이 공백인 경우 같은 시간에 방송된 상품의 노출(분)으로 채워 결측치를 없앤 변수 
    #(2) 누적 노출(분) 파생 변수 생성  
    #(3) 마감시간대 여부 관련 파생 변수 생성    
  
    노출분 = list(실적['노출(분)'])
 
    for idx, i in zip(실적.index, 노출분):
        if math.isnan(i) == True:
            노출분[idx] = 노출분[idx-1]

    실적['노출분'] = 노출분

    # 누적 노출 변수 : 각 상품에 대한 방송 듀레이션을 누적하여 더한 방송 노출 시간 (분) 
    # 마감 시간 여부 : 각 상품의 방송이 끝나는 시간을 1 로, 방송 시작 ~ 방송 중일 때는 0 으로 표시한 변수 
    누적노출 = []
    마감여부 = []
    cnt = 1
 
    for i in 실적.index:
        if i == 0:
            item_temp = 실적.상품코드[i]
            time_temp = 실적['노출(분)'][i]
            ex = 실적['노출(분)'][i]
            누적노출.append(ex)
            continue
      
        a,b = 실적.방송일시[i], 실적.방송일시[i-1]
      
        if a == b:
            cnt+=1
            누적노출.append(ex)
      
        elif a != b:
            if 실적.상품코드[i] == item_temp:
                ex += time_temp
                누적노출.append(ex)
                마감여부.extend([0 for i in range(cnt)])
                cnt=1
            else:
                item_temp = 실적.상품코드[i]
                time_temp = 실적['노출(분)'][i]
                ex = 실적['노출(분)'][i]
                누적노출.append(ex)
                마감여부.extend([1 for i in range(cnt)])
                cnt=1

        if i == (len(실적.index)-1):
            마감여부.extend([1 for i in range(cnt)])

    실적['누적노출'] = 누적노출
    실적['마감시간여부'] =마감여부

    #무형인 상품군 제외 

    실적 = 실적[실적.상품군!='무형']
    실적.reset_index(inplace=True, drop=True)

    #취급액이 공백인 경우 0으로 채우기
    
    실적.취급액 = 실적.취급액.fillna(0)

    # 월 변수에서 '계절' 파생변수 생성
    # 계절_categoric : 계절 이름을 컬럼 값으로 가짐
    # 계절_numeric : 계절 이름을 순서대로 숫자 1 ~ 4 에 대응시킨 값을 가짐
    
    계절_numeric = []
    계절_categoric = []
    
    for i in 실적.월:

        if (i==3)|(i==4)|(i==5):
            계절_numeric.append(1)
            계절_categoric.append('spring')

        elif (i==6)|(i==7)|(i==8):
            계절_numeric.append(2)
            계절_categoric.append('summer')

        elif (i==9)|(i==10)|(i==11):
            계절_numeric.append(3)
            계절_categoric.append('autumn')

        elif (i==12)|(i==1)|(i==2):
            계절_numeric.append(4)
            계절_categoric.append('winter')

    실적['계절_numeric'] = 계절_numeric
    실적['계절_categoric'] = 계절_categoric

    return 실적

In [6]:
# 2019년 판매 실적 데이터 전처리 
실적 = preprocessing_table_2019(실적)

In [7]:
# 결과 확인
실적.head(50)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,년,주차,...,분,주말,방송날짜,공휴일,지불방식,노출분,누적노출,마감시간여부,계절_numeric,계절_categoric
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,2019,1,...,0,0,2019-01-01,1,0,20.0,20.0,0,4,winter
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,2019,1,...,0,0,2019-01-01,1,0,20.0,20.0,0,4,winter
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,2019,1,...,20,0,2019-01-01,1,0,20.0,40.0,0,4,winter
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,2019,1,...,20,0,2019-01-01,1,0,20.0,40.0,0,4,winter
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,2019,1,...,40,0,2019-01-01,1,0,20.0,60.0,1,4,winter
5,2019-01-01 06:40:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,9337000.0,2019,1,...,40,0,2019-01-01,1,0,20.0,60.0,1,4,winter
6,2019-01-01 07:00:00,20.0,100305,200974,오모떼 레이스 파운데이션 브라,속옷,59000,6819000.0,2019,1,...,0,0,2019-01-01,1,0,20.0,20.0,0,4,winter
7,2019-01-01 07:20:00,20.0,100305,200974,오모떼 레이스 파운데이션 브라,속옷,59000,15689000.0,2019,1,...,20,0,2019-01-01,1,0,20.0,40.0,0,4,winter
8,2019-01-01 07:40:00,20.0,100305,200974,오모떼 레이스 파운데이션 브라,속옷,59000,25370000.0,2019,1,...,40,0,2019-01-01,1,0,20.0,60.0,1,4,winter
9,2019-01-01 08:00:00,20.0,100808,202377,CERINI by PAT 남성 소프트 기모 릴렉스팬츠,의류,59900,16133000.0,2019,1,...,0,0,2019-01-01,1,0,20.0,20.0,0,4,winter


In [8]:
실적.tail(3)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,년,주차,...,분,주말,방송날짜,공휴일,지불방식,노출분,누적노출,마감시간여부,계절_numeric,계절_categoric
37369,2020-01-01,NaN,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,2020,1,...,0,0,2020-01-01,1,2,20.0,60.0,1,4,winter
37370,2020-01-01,NaN,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,2020,1,...,0,0,2020-01-01,1,1,20.0,60.0,1,4,winter
37371,2020-01-01,NaN,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,46608000.0,2020,1,...,0,0,2020-01-01,1,2,20.0,60.0,1,4,winter


### 1) - 2. 2020년도 6월 평가 데이터 전처리 

In [9]:
def preprocessing_table_2020(평가):
    '''
    평가 데이터 전처리 및 파생변수 생성하는 작업 수행 
    '''
  
    #방송일시 변수에서 파생 변수 생성
    
    평가['년'] = 평가.방송일시.apply(lambda X: X.year)
    평가['주차'] = 평가.방송일시.apply(lambda X: X.week)
    평가['월'] = 평가.방송일시.apply(lambda X: X.month)
    평가['일'] = 평가.방송일시.apply(lambda X: X.day)
    평가['요일'] = 평가.방송일시.apply(lambda X: X.weekday()) # 0 - 월요일, 1 - 화요일, ..., 6 - 일요일
    평가['시'] = 평가.방송일시.apply(lambda X: X.hour)
    평가['분'] = 평가.방송일시.apply(lambda X: X.minute)
    평가['주말'] = [1 if i==5 or i==6 else 0 for i in 평가['요일']]
    평가['방송날짜'] = 평가.방송일시.dt.date # '방송일시' 컬럼에서 시/분 제외한 날짜 정보 추출하여 별도의 컬럼 생성
  
    # 2020년도 6월 기준 공휴일 정보를 바탕으로 별도의 컬럼 생성
    # 주말과 겹치는 공휴일도 상관없이 표시
    # 공휴일인 날은 1, 공휴일 아닌 날은 0
    
    평가['공휴일'] = [i.strftime('%Y-%m-%d') for i in 평가.방송날짜]
    
    공휴일 = [
                '2020-06-06'
            ]
    idx = 평가.공휴일.isin(공휴일)
    nidx = ~ 평가.공휴일.isin(공휴일)
    평가.loc[idx,'공휴일'] = 1
    평가.loc[nidx,'공휴일'] = 0

    # '상품명' 기준으로 '지불방식' 에 따른 파생 변수 생성
    
    지불방식 = []
    for i in 평가.상품명:
        if '무이자' in i or '(무)' in i: # 상품명에 '무이자' 라는 표기 있을 시 1 대입
            지불방식.append(1)
        elif '일시불' in i or '(일)' in i: # 상품명에 '일시불' 이라는 표기 있을 시 2 대입
            지불방식.append(2)
        else:
            지불방식.append(0) # 상품명에 지불 방식 관련 표기 없을 시 0 대입
 
    평가['지불방식'] = 지불방식

    #노출(분) 변수로 파생 변수 생성 
    #(1) 노출분 : 노출(분)이 공백인 경우 같은 시간에 방송된 상품의 노출(분)으로 채워 결측치를 없앤 변수 
    #(2) 누적 노출(분) 파생 변수 생성  
    #(3) 마감시간대 여부 관련 파생 변수 생성    
  
    노출분 = list(평가['노출(분)'])
 
    for idx, i in zip(평가.index, 노출분):
        if math.isnan(i) == True:
            노출분[idx] = 노출분[idx-1]

    평가['노출분'] = 노출분

    # 누적 노출 변수 : 각 상품에 대한 방송 듀레이션을 누적하여 더한 방송 노출 시간 (분) 
    # 마감 시간 여부 : 각 상품의 방송이 끝나는 시간을 1 로, 방송 시작 ~ 방송 중일 때는 0 으로 표시한 변수 
    누적노출 = []
    마감여부 = []
    cnt = 1
 
    for i in 평가.index:
        if i == 0:
            item_temp = 평가.상품코드[i]
            time_temp = 평가['노출(분)'][i]
            ex = 평가['노출(분)'][i]
            누적노출.append(ex)
            continue
      
        a,b = 평가.방송일시[i], 평가.방송일시[i-1]
      
        if a == b:
            cnt+=1
            누적노출.append(ex)
      
        elif a != b:
            if 평가.상품코드[i] == item_temp:
                ex += time_temp
                누적노출.append(ex)
                마감여부.extend([0 for i in range(cnt)])
                cnt=1
            else:
                item_temp = 평가.상품코드[i]
                time_temp = 평가['노출(분)'][i]
                ex = 평가['노출(분)'][i]
                누적노출.append(ex)
                마감여부.extend([1 for i in range(cnt)])
                cnt=1

        if i == (len(평가.index)-1):
            마감여부.extend([1 for i in range(cnt)])

    평가['누적노출'] = 누적노출
    평가['마감시간여부'] =마감여부

    #무형인 상품군 제외 

    평가 = 평가[평가.상품군!='무형']
    평가.reset_index(inplace=True, drop=True)

    #취급액이 공백인 경우 0으로 채우기 
    
    평가.취급액 = 평가.취급액.fillna(0)

    # 월 변수에서 '계절' 파생변수 생성
    # 계절_categoric : 계절 이름을 컬럼 값으로 가짐
    # 계절_numeric : 계절 이름을 순서대로 숫자 1 ~ 4 에 대응시킨 값을 가짐
    
    계절_numeric = []
    계절_categoric = []
    
    for i in 평가.월:

        if (i==3)|(i==4)|(i==5):
            계절_numeric.append(1)
            계절_categoric.append('spring')

        elif (i==6)|(i==7)|(i==8):
            계절_numeric.append(2)
            계절_categoric.append('summer')

        elif (i==9)|(i==10)|(i==11):
            계절_numeric.append(3)
            계절_categoric.append('autumn')

        elif (i==12)|(i==1)|(i==2):
            계절_numeric.append(4)
            계절_categoric.append('winter')

    평가['계절_numeric'] = 계절_numeric
    평가['계절_categoric'] = 계절_categoric

    return 평가

In [10]:
# 2020년 6월 평가 데이터 전처리
평가 = preprocessing_table_2020(평가)

In [11]:
# 결과 확인
평가.head(3)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,년,주차,...,분,주말,방송날짜,공휴일,지불방식,노출분,누적노출,마감시간여부,계절_numeric,계절_categoric
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,0.0,2020,23,...,20,0,2020-06-01,0,0,20.0,20.0,0,2,summer
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,0.0,2020,23,...,40,0,2020-06-01,0,0,20.0,40.0,0,2,summer
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,0.0,2020,23,...,0,0,2020-06-01,0,0,20.0,60.0,1,2,summer


## 2) 외부 데이터 병합

활용한 외부 데이터

(1) 날씨 데이터 
*   기온, 습도, 풍속, 강수량, 태풍 데이터 (전국 단위 기준)

(2) 네이버 검색어 트렌드 데이터
*   홈쇼핑 트렌드) 네이버에서 '홈쇼핑', 'NS홈쇼핑'검색량 
*   상품군 별 트렌드) 네이버에서 각 상품군 클릭률 





### 2) - 1. 2019년도 실적 데이터와 외부 데이터 병합

In [12]:
def merge_external_data_2019(실적):
    '''
    판매실적 데이터에 외부 데이터를 병합하는 작업 수행
    '''
    # (1)날씨 데이터 병합

    날씨데이터 = pd.read_excel('../data/external/weather/날씨데이터.xlsx', sheet_name =None)
    태풍, 일별기온, 일별강수량 = 날씨데이터.values()
    일별기온.columns = ['날짜', '지점', '평균기온', '최저기온', '최고기온']

    일별기온.drop('지점',axis=1,inplace=True)
    일별강수량.drop('지점',axis=1,inplace=True)
    일별기온.columns = ['날짜','일평균기온','일최저기온','일최고기온']
    일별강수량.columns = ['날짜','일별강수량']
    일별기온.날짜 = [i.strftime('%Y-%m-%d') for i in 일별기온.날짜]
    실적.방송날짜 = [i.strftime('%Y-%m-%d') for i in 실적.방송날짜] # 혹시 실적 데이터의 방송날짜 타입이 datetime 일 경우 uncomment
    일별강수량.날짜 = [i.strftime('%Y-%m-%d') for i in 일별강수량.날짜]
    
    실적 = pd.merge(실적,일별기온,left_on='방송날짜',right_on='날짜',how='left')
    del 실적['날짜']
    실적 = pd.merge(실적,일별강수량,left_on='방송날짜',right_on='날짜',how='left')
    del 실적['날짜']
    
    # '태풍 발생' 변수 생성 : 태풍 발생 ~ 소멸까지의 날짜는 1로 표시, 아닌 날짜는 0 으로 표시 
    
    발생일시 = []
    소멸일시 = []

    for i in 태풍.발생_소멸:
        발생일시.append(dt.datetime.strptime(i[:10]+str('/')+i[11:13], '%Y/%m/%d/%H'))
        소멸일시.append(dt.datetime.strptime(i[19:29]+str('/')+i[30:32], '%Y/%m/%d/%H'))
      
    태풍['발생일시'] = 발생일시
    태풍['소멸일시'] = 소멸일시

    태풍발생 = [0]*(len(실적))
    for i in range(0, len(태풍)):
        for n in 실적[(태풍.발생일시[i] <= 실적.방송일시) & (태풍.소멸일시[i] >= 실적.방송일시)].index:
            태풍발생[n] = 1
    
    실적['태풍발생'] = 태풍발생
    
    # 전국 일평균 습도와 풍속 데이터 병합 
    # 일평균 습도와 풍속 raw data 를 가공한 데이터 이용, 가공 방법은 발표 자료의 부록 참조 바람

    습도풍속 = pd.read_csv('../data/external/weather/humidity_wind_2019.csv')
    습도풍속.columns = ['일시', '일별상대습도', '일평균풍속']
    실적 = pd.merge(실적, 습도풍속, left_on='방송날짜', right_on='일시', how= 'left')
    del 실적['일시']

    # (2) 홈쇼핑 검색어 트렌드 데이터 병합
    
    NSshop = pd.read_excel('../data/external/naver_trend/홈쇼핑.xlsx')
    NSshop.columns = ['날짜','홈쇼핑검색량','NS검색량']

    실적 = 실적.merge(NSshop,left_on='방송날짜',right_on='날짜',how='left')
    del 실적['날짜']
    
    return 실적

In [13]:
# 2019 판매 데이터에 외부 데이터 병합 (상품군 별 네이버 트렌드 데이터는 전처리 필요하여 아래에서 따로 병합)
실적 = merge_external_data_2019(실적)

In [14]:
# 결과 확인
pd.options.display.max_columns = 40
실적.head(3)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,년,주차,월,일,요일,시,분,주말,방송날짜,공휴일,지불방식,노출분,누적노출,마감시간여부,계절_numeric,계절_categoric,일평균기온,일최저기온,일최고기온,일별강수량,태풍발생,일별상대습도,일평균풍속,홈쇼핑검색량,NS검색량
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,2019,1,1,1,1,6,0,0,2019-01-01,1,0,20.0,20.0,0,4,winter,-2.1,-5.8,2.1,0.1,0,57.726667,2.535556,28.12213,0.65648
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,2019,1,1,1,1,6,0,0,2019-01-01,1,0,20.0,20.0,0,4,winter,-2.1,-5.8,2.1,0.1,0,57.726667,2.535556,28.12213,0.65648
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,2019,1,1,1,1,6,20,0,2019-01-01,1,0,20.0,40.0,0,4,winter,-2.1,-5.8,2.1,0.1,0,57.726667,2.535556,28.12213,0.65648


In [15]:
# 네이버 검색어 트렌드 데이터 중 상품군 별 데이터 2차 가공 

# 농수축 상품군 내 농산, 수산, 축산, 가공식품, 김치 등 세부 카테고리 중 김치만 클릭율 변화 추이가 다름
# 따라서 김치 상품의 클릭율 데이터를 구분하여 실적 데이터에 병합할 필요성 존재
# 농수축에서 김치만 따로 상품군 재정의 하기 

농수축 = 실적[실적['상품군'] == '농수축']
식품_소분류 = [ ]  
식품_소분류 = np.select(
    condlist=[ 
        농수축['상품명'].str.contains('김치')
    ],
    choicelist=['김치'], # dummy variables
    default= '농수축'#  없음 
)
# 농수축,가공 식품은 0 으로 표시, 김치는 1로 표시

농수축['상품군'] = 식품_소분류

실적 = 실적.set_index(['방송일시','상품명'])
농수축 = 농수축.set_index(['방송일시','상품명'])
실적.update(농수축)
실적.reset_index(inplace=True)

In [16]:
# (3) 상품군 별 네이버 트렌드 데이터 (클릭율) 병합

# 병합 전 상품군 별 네이버 트렌드 데이터 전처리

path = "../data/external/naver_trend/2019/상품군_트렌드/"
file_list = os.listdir(u"../data/external/naver_trend/2019/상품군_트렌드/")
file_list = [normalize('NFC',i) for i in file_list]


trend_files={}

for i in file_list:
    f = path+i
    idx = copy.deepcopy(i.split('.')[0])
    trend_files[idx] = pd.read_csv(f, engine = 'python')
    

temp = {}
for i in list(trend_files.keys()):
    if len(trend_files[i].columns) > 2:
        temp[i] = trend_files[i]

for i in list(temp.keys()):
    temp[i][i[:-4]] = temp[i].iloc[:,1:].mean(axis=1)

for i in list(temp.keys()):
    trend_files[i] = temp[i][['날짜',i[:-4]]]

for i in list(trend_files.keys()):
    trend_files[i].columns = ['날짜',i[:-4]]

# 2030 세대와 4050 세대의 클릭율 데이터를 3:7 비율로 가중평균하여 병합

last_file = {}
k = np.unique([i[:-4] for i in list(trend_files.keys())])
for i in k:
    a = pd.DataFrame()
    a['날짜'] = trend_files[i+'2030'].날짜
    a[i] = trend_files[i+'2030'][i].apply(float)*0.3 + trend_files[i+'4050'][i].apply(float)*0.7
    last_file[i] = a

# 가중평균 낸 클릭율 값을 정수로 반올림

for i in list(last_file.keys()):
    last_file[i][i] = (last_file[i][i]*(100/max(last_file[i][i]))).round(0)
    
# 실적 데이터에 가공한 클릭율 데이터 병합

for i in list(last_file.keys()):
    last_file[i]['날짜_상품군'] = [j+'_'+i for j in last_file[i].날짜]
    last_file[i].columns = ['날짜','클릭비율','날짜_상품군']

실적['날짜_상품군'] = [실적.방송날짜.values[i]+'_'+실적.상품군.values[i] for i in range(len(실적))]

real_last = pd.DataFrame({'날짜':[],'클릭비율':[],'날짜_상품군':[]})
for i in list(last_file.keys()):
    real_last = pd.concat([real_last,last_file[i]])

del real_last['날짜']
실적 = 실적.merge(real_last,on='날짜_상품군',how='left')
del 실적['날짜_상품군']

In [17]:
# 상품군 별 네이버 트렌드 데이터까지 병합한 결과 확인 
pd.options.display.max_columns = 40
실적.head(3)

,방송일시,상품명,노출(분),마더코드,상품코드,상품군,판매단가,취급액,년,주차,월,일,요일,시,분,주말,방송날짜,공휴일,지불방식,노출분,누적노출,마감시간여부,계절_numeric,계절_categoric,일평균기온,일최저기온,일최고기온,일별강수량,태풍발생,일별상대습도,일평균풍속,홈쇼핑검색량,NS검색량,클릭비율
0,2019-01-01 06:00:00,테이트 남성 셀린니트3종,20.0,100346.0,201072.0,의류,39900.0,2099000.0,2019.0,1.0,1.0,1.0,1.0,6.0,0.0,0.0,2019-01-01,1,0.0,20.0,20.0,0.0,4.0,winter,-2.1,-5.8,2.1,0.1,0.0,57.726667,2.535556,28.12213,0.65648,49.0
1,2019-01-01 06:00:00,테이트 여성 셀린니트3종,NaN,100346.0,201079.0,의류,39900.0,4371000.0,2019.0,1.0,1.0,1.0,1.0,6.0,0.0,0.0,2019-01-01,1,0.0,20.0,20.0,0.0,4.0,winter,-2.1,-5.8,2.1,0.1,0.0,57.726667,2.535556,28.12213,0.65648,49.0
2,2019-01-01 06:20:00,테이트 남성 셀린니트3종,20.0,100346.0,201072.0,의류,39900.0,3262000.0,2019.0,1.0,1.0,1.0,1.0,6.0,20.0,0.0,2019-01-01,1,0.0,20.0,40.0,0.0,4.0,winter,-2.1,-5.8,2.1,0.1,0.0,57.726667,2.535556,28.12213,0.65648,49.0


### 2) - 2. 평가 데이터에 외부 데이터 병합 

In [18]:
def merge_external_data_2020(평가):
    '''
    평가 데이터에 외부 데이터를 병합하는 작업 수행
    '''
    # (1)날씨 데이터 병합

    날씨데이터 = pd.read_excel('../data/external/weather/날씨데이터.xlsx', sheet_name =None)
    태풍, 일별기온, 일별강수량 = 날씨데이터.values()
    일별기온.columns = ['날짜', '지점', '평균기온', '최저기온', '최고기온']

    일별기온.drop('지점',axis=1,inplace=True)
    일별강수량.drop('지점',axis=1,inplace=True)
    일별기온.columns = ['날짜','일평균기온','일최저기온','일최고기온']
    일별강수량.columns = ['날짜','일별강수량']
    일별기온.날짜 = [i.strftime('%Y-%m-%d') for i in 일별기온.날짜]
    평가.방송날짜 = [i.strftime('%Y-%m-%d') for i in 평가.방송날짜] # 혹시 평가 데이터의 방송날짜 타입이 datetime 일 경우 uncomment
    일별강수량.날짜 = [i.strftime('%Y-%m-%d') for i in 일별강수량.날짜]
    
    평가 = pd.merge(평가, 일별기온,left_on='방송날짜',right_on='날짜', how='left')
    del 평가['날짜']
    평가 = pd.merge(평가, 일별강수량,left_on='방송날짜',right_on='날짜', how='left')
    del 평가['날짜']
    
    # '태풍 발생' 변수 생성 : 태풍 발생 ~ 소멸까지의 날짜는 1로 표시, 아닌 날짜는 0 으로 표시 
    
    발생일시 = []
    소멸일시 = []

    for i in 태풍.발생_소멸:
        발생일시.append(dt.datetime.strptime(i[:10]+str('/')+i[11:13], '%Y/%m/%d/%H'))
        소멸일시.append(dt.datetime.strptime(i[19:29]+str('/')+i[30:32], '%Y/%m/%d/%H'))
      
    태풍['발생일시'] = 발생일시
    태풍['소멸일시'] = 소멸일시

    태풍발생 = [0]*(len(평가))
    for i in range(0, len(태풍)):
        for n in 평가[(태풍.발생일시[i] <= 평가.방송일시) & (태풍.소멸일시[i] >= 평가.방송일시)].index:
            태풍발생[n] = 1
    
    평가['태풍발생'] = 태풍발생
    
    # 전국 일평균 습도와 풍속 데이터 병합 
    # 일평균 습도와 풍속 raw data 를 가공한 데이터 이용, 가공 방법은 발표 자료의 부록 참조 바람

    습도풍속 = pd.read_csv('../data/external/weather/humidity_wind_2020.csv')
    습도풍속.columns = ['일시', '일별상대습도', '일평균풍속']
    평가 = pd.merge(평가, 습도풍속, left_on='방송날짜', right_on='일시', how = 'left')
    del 평가['일시']

    # (2) 홈쇼핑 검색어 트렌드 데이터 병합
    
    NSshop = pd.read_excel('../data/external/naver_trend/홈쇼핑.xlsx')
    NSshop.columns = ['날짜','홈쇼핑검색량','NS검색량']

    평가 = 평가.merge(NSshop,left_on = '방송날짜',right_on = '날짜',how='left')
    del 평가['날짜']
    
    return 평가

In [19]:
# 2020 평가 데이터에 외부 데이터 병합 (2020 상품군 별 네이버 트렌드 데이터는 가공 필요하여 아래에서 따로 병합)
평가 = merge_external_data_2020(평가)

In [20]:
# 결과 확인
평가.head(3)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,년,주차,월,일,요일,시,분,주말,방송날짜,공휴일,지불방식,노출분,누적노출,마감시간여부,계절_numeric,계절_categoric,일평균기온,일최저기온,일최고기온,일별강수량,태풍발생,일별상대습도,일평균풍속,홈쇼핑검색량,NS검색량
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,0.0,2020,23,6,1,0,6,20,0,2020-06-01,0,0,20.0,20.0,0,2,summer,20.8,15.8,26.0,0.3,0,62.12,2.602222,23.92366,2.0916
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,0.0,2020,23,6,1,0,6,40,0,2020-06-01,0,0,20.0,40.0,0,2,summer,20.8,15.8,26.0,0.3,0,62.12,2.602222,23.92366,2.0916
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,0.0,2020,23,6,1,0,7,0,0,2020-06-01,0,0,20.0,60.0,1,2,summer,20.8,15.8,26.0,0.3,0,62.12,2.602222,23.92366,2.0916


In [21]:
# 네이버 검색어 트렌드 데이터 중 상품군 별 데이터 2차 가공 

# 농수축 상품군 내 농산, 수산, 축산, 가공식품, 김치 등 세부 카테고리 중 김치만 클릭율 변화 추이가 다름
# 따라서 김치 상품의 클릭율 데이터를 구분하여 실적 데이터에 병합할 필요성 존재
# 농수축에서 김치만 따로 상품군 재정의 하기 

농수축 = 평가[평가['상품군'] == '농수축']
식품_소분류 = [ ]  
식품_소분류 = np.select(
    condlist=[ 
        농수축['상품명'].str.contains('김치')
    ],
    choicelist=['김치'], # dummy variables
    default= '농수축'#  없음 
)
# 농수축,가공 식품은 0 으로 표시, 김치는 1로 표시

농수축['상품군'] = 식품_소분류

평가 = 평가.set_index(['방송일시','상품명'])
농수축 = 농수축.set_index(['방송일시','상품명'])
평가.update(농수축)
평가.reset_index(inplace=True)

In [22]:
## (3) 상품군 별 네이버 트렌드 데이터 (클릭율) 병합

# 병합 전 상품군 별 네이버 트렌드 데이터 전처리

path = "../data/external/naver_trend/2020/상품군_트렌드/"
file_list = os.listdir(u"../data/external/naver_trend/2020/상품군_트렌드/")
file_list = [normalize('NFC',i) for i in file_list]

trend_files={}

for i in file_list:
    f = path+i
    idx = copy.deepcopy(i.split('.')[0])
    trend_files[idx] = pd.read_csv(f, engine = 'python')


temp = {}
for i in list(trend_files.keys()):
    if len(trend_files[i].columns) > 2:
        temp[i] = trend_files[i]

for i in list(temp.keys()):
    temp[i][i[:-4]] = temp[i].iloc[:,1:].mean(axis=1)

for i in list(temp.keys()):
    trend_files[i] = temp[i][['날짜',i[:-4]]]

for i in list(trend_files.keys()):
    trend_files[i].columns = ['날짜',i[:-4]]

# 2030 세대와 4050 세대의 클릭율 데이터를 3:7 비율로 가중평균하여 병합

last_file2 = {}
k = np.unique([i[:-4] for i in list(trend_files.keys())])
for i in k:
    a = pd.DataFrame()
    a['날짜'] = trend_files[i+'2030'].날짜
    a[i] = trend_files[i+'2030'][i].apply(float)*0.3 + trend_files[i+'4050'][i].apply(float)*0.7
    last_file2[i] = a

# 가중평균 낸 클릭율 값을 정수로 반올림

for i in list(last_file2.keys()):
    last_file2[i][i] = (last_file2[i][i]*(100/max(last_file2[i][i]))).round(0)
    


for i in list(last_file2.keys()):
    last_file2[i]['날짜_상품군'] = [j+'_'+i for j in last_file2[i].날짜]
    last_file2[i].columns = ['날짜','클릭비율','날짜_상품군']

In [23]:
for i in list(last_file2.keys()):
    value = last_file[i]['클릭비율'][365] # 2019년도 기준으로 뽑은 데이터를 통해 보정치 설정 (각 상품군 별 마지막 레코드)
    for a in range(len(last_file2[i])):
        if last_file2[i]['날짜'][a]== '2020-01-01':
            x = last_file2[i]['클릭비율'][a] / value # 보정 비율 x
            last_file2[i]['클릭비율'][a] = last_file2[i]['클릭비율'][a] * x

In [24]:
# 보정된 클릭율 데이터 병합

평가['날짜_상품군'] = [평가.방송날짜.values[i]+'_'+평가.상품군.values[i] for i in range(len(평가))]

real_last = pd.DataFrame({'날짜':[],'클릭비율':[],'날짜_상품군':[]})

for i in list(last_file2.keys()):
    real_last = pd.concat([real_last,last_file2[i]])


del real_last['날짜']
평가 = 평가.merge(real_last,on='날짜_상품군',how='left')
del 평가['날짜_상품군']

## 전처리 완료한 데이터 따로 저장 

In [25]:
# 2019 실적 데이터 마지막 확인
실적.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37372 entries, 0 to 37371
Data columns (total 34 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   방송일시          37372 non-null  datetime64[ns]
 1   상품명           37372 non-null  object        
 2   노출(분)         20588 non-null  float64       
 3   마더코드          37372 non-null  float64       
 4   상품코드          37372 non-null  float64       
 5   상품군           37372 non-null  object        
 6   판매단가          37372 non-null  float64       
 7   취급액           37372 non-null  float64       
 8   년             37372 non-null  float64       
 9   주차            37372 non-null  float64       
 10  월             37372 non-null  float64       
 11  일             37372 non-null  float64       
 12  요일            37372 non-null  float64       
 13  시             37372 non-null  float64       
 14  분             37372 non-null  float64       
 15  주말            37372 non-null  float6

In [26]:
# 2020 평가 데이터 마지막 확인
평가.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2716 entries, 0 to 2715
Data columns (total 34 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   방송일시          2716 non-null   datetime64[ns]
 1   상품명           2716 non-null   object        
 2   노출(분)         1617 non-null   float64       
 3   마더코드          2716 non-null   float64       
 4   상품코드          2716 non-null   float64       
 5   상품군           2716 non-null   object        
 6   판매단가          2716 non-null   float64       
 7   취급액           2716 non-null   float64       
 8   년             2716 non-null   float64       
 9   주차            2716 non-null   float64       
 10  월             2716 non-null   float64       
 11  일             2716 non-null   float64       
 12  요일            2716 non-null   float64       
 13  시             2716 non-null   float64       
 14  분             2716 non-null   float64       
 15  주말            2716 non-null   float64 

In [27]:
# 바로 모델링에 사용 가능하도록 실적 데이터에서 취급액이 0 인 데이터는 제외하고 저장
실적_전처리 = 실적[실적.취급액 != 0]

실적_전처리.to_excel('../data/internal/preprocessed_data/train_data/실적_전처리.xlsx', index = False)
평가.to_excel('../data/internal/preprocessed_data/test_data/평가_전처리.xlsx', index = False)